In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from textblob import TextBlob
from flair.models import TextClassifier
from flair.data import Sentence
import torch
from senticnet.senticnet import SenticNet 


In [ ]:
import pandas as pd
from tqdm import tqdm
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Initialize SenticNet
senticnet = SenticNet()

def get_senticnet_sentiment(text):
    try:
        
        return float(senticnet.polarity_value(text))
    except KeyError:
       
        return 0.0

In [ ]:
# Initialize models
vader_analyzer = SentimentIntensityAnalyzer()
flair_lstm = TextClassifier.load('sentiment-fast').to('cuda')  
flair_bert = TextClassifier.load('sentiment').to('cuda')  



In [ ]:
def analyze_sentiment_with_gpu(tweet):
   
    vader_score = vader_analyzer.polarity_scores(tweet)['compound']
    
   
    textblob_score = TextBlob(tweet).sentiment.polarity
    
  
    lstm_sentence = Sentence(tweet)
    flair_lstm.predict(lstm_sentence)
    lstm_score = lstm_sentence.labels[0].score * (1 if "POSITIVE" in str(lstm_sentence.labels[0]) else -1)
    
  
    bert_sentence = Sentence(tweet)
    flair_bert.predict(bert_sentence)
    bert_score = bert_sentence.labels[0].score * (1 if "POSITIVE" in str(bert_sentence.labels[0]) else -1)

    
    senticnet_score = get_senticnet_sentiment(tweet)
    
    
    combined_score = (0.3*vader_score + 0.3*textblob_score + 0.85*lstm_score + 0.85*bert_score + 0.4*senticnet_score) / 5
    
 
    if combined_score < -0.35:
        sentiment_label = "Negative"
    elif combined_score > 0.35:
        sentiment_label = "Positive"
    else:
        sentiment_label = "Neutral"
    
    return {
        "VADER": vader_score,
        "TextBlob": textblob_score,
        "Flair_LSTM": lstm_score,
        "Flair_BERT": bert_score,
        "SenticNet": senticnet_score,
        "Combined_Score": combined_score,
        "Sentiment_Label": sentiment_label
    }


In [ ]:

def process_file(file_path, output_path):
    
    df = pd.read_csv(file_path)
    
    
    if 'cleaned_tweet' not in df.columns:
        raise ValueError("The file must contain a 'cleaned_tweet' column.")
    
    
    tqdm.pandas(desc="Analyzing Sentiments")
    
    
    df['sentiment'] = df['cleaned_tweet'].progress_apply(analyze_sentiment_with_gpu)
    
   
    df.to_csv(output_path, index=False)
    print(f"Sentiment analysis completed. Results saved to {output_path}.")


file_path = "/content/drive/MyDrive/preprocessed_data/predicted_genders_filtered.csv"  
output_path = "/content/drive/MyDrive/preprocessed_data/predicted_sentiment.csv"  #
process_file(file_path, output_path)